Implementation of the problem outlined in <https://www.reddit.com/r/Python/comments/7awlm1/is_this_something_which_is_possible_in_python/>

## Setup

In [1]:
from z3 import Int, Solver, If, And, Sum

## Example data

In [2]:
## A very obvious toy example
## 3 rooms, 7 (male) + 11 (female) participants
## The obvious solution is to put all males together, and divide the females over the other two rooms

roomsizes = [4, 7, 7]

# data for each participant:
# tuple of (gender, tuple of "buddies")
participants = [
    # males
    ("m", (1, 2, 3)),
    ("m", (0, 4, 5)),
    ("m", (0, 4, 5)),
    ("m", (1, 2, 4)),
    ("m", (0, 1, 2)),
    ("m", (0, 1, 2)),
    ("m", (0, 1, 2)),
    # females
    ("f", (8, 9, 10)),
    ("f", (7, 9, 10)),
    ("f", (7, 8, 13)),
    ("f", (7, 8, 9)),
    ("f", (7, 8, 9)),
    ("f", (7, 8, 9)),
    ("f", (7, 8, 9)),
    ("f", (7, 10, 11)),
    ("f", (8, 9, 13)),
    ("f", (8, 12, 14)),
    ("f", (8, 15, 16)),
]

### summary of example data

In [3]:
Nrooms = len(roomsizes)

In [4]:
Nparticipants = len(participants)

Nmale = sum((1 for p in participants if p[0] == "m"))
Nfemale = sum((1 for p in participants if p[0] == "f"))

Nparticipants, Nmale, Nfemale

(18, 7, 11)

### Sanity checks

The number of available beds is at least equal to the number of participants

In [5]:
assert Nparticipants <= sum(roomsizes)

Every participant didn't choose themselves as a buddy, nor someone of the opposite sex:

In [6]:
for ip, p in enumerate(participants):
    gender = p[0]
    buddies = p[1]
    assert not ip in buddies, "Participant %d has themselves as buddy" % ip
    for ib in buddies:
        assert participants[ib][0] == gender, "Buddy %d of participant %d has opposite sex" % (ib, ip)

---

## Setup Z3 variables

These "Int" variables will hold the (0-based) room index for each participant

In [7]:
z3participants = [Int("participant_%d" % ip) for ip in range(Nparticipants)]

---

## Define constraints

### Constraint: each participant is assigned a valid room

In [8]:
inroom_c = [
    And(
        p >= 0,
        p < Nrooms,
    )
    for p in z3participants
]

### Constraint: the number of participants per room does not exceed the capacity of the room

In [9]:
roomsize_c = [
    Sum([If(p == i, 1, 0) for p in z3participants]) <= roomsizes[i]
    for i in range(Nrooms)
]

### Constraint: participants per room are either all female or all male

or: no equal room number for all of the possible pairs of male:female participants

In [10]:
gender_c = [
    z3participants[ip_male] != z3participants[ip_female]
    for ip_male in (ip for (ip, p) in enumerate(participants) if p[0] == "m")
    for ip_female in (ip for (ip, p) in enumerate(participants) if p[0] == "f")
]

### Constraint: each participant is assigned to a room where at least 1 of their chosen participants is also assigned to

ie the "buddy" constraint

In [11]:
buddy_c = [
    Sum([
        If(z3participants[ip] == z3participants[ibuddy], 1, 0)
        for ibuddy in participants[ip][1]
    ]) >= 1
    for ip in range(Nparticipants)
]

---

## Solve

In [12]:
solver = Solver()
solver.add(inroom_c)
solver.add(roomsize_c)
solver.add(gender_c)
solver.add(buddy_c)

In [13]:
solver.check()

sat

In [14]:
mdl = solver.model()

In [15]:
list(zip(range(Nparticipants), participants, [mdl.evaluate(p) for p in z3participants]))

[(0, ('m', (1, 2, 3)), 1),
 (1, ('m', (0, 4, 5)), 1),
 (2, ('m', (0, 4, 5)), 1),
 (3, ('m', (1, 2, 4)), 1),
 (4, ('m', (0, 1, 2)), 1),
 (5, ('m', (0, 1, 2)), 1),
 (6, ('m', (0, 1, 2)), 1),
 (7, ('f', (8, 9, 10)), 2),
 (8, ('f', (7, 9, 10)), 2),
 (9, ('f', (7, 8, 13)), 0),
 (10, ('f', (7, 8, 9)), 2),
 (11, ('f', (7, 8, 9)), 2),
 (12, ('f', (7, 8, 9)), 0),
 (13, ('f', (7, 8, 9)), 0),
 (14, ('f', (7, 10, 11)), 2),
 (15, ('f', (8, 9, 13)), 2),
 (16, ('f', (8, 12, 14)), 0),
 (17, ('f', (8, 15, 16)), 2)]

*All males go to room 1*

*4 females go to room 0, the others to room 2*

---